In [1]:
import os
import google.generativeai as genai
from dotenv import load_dotenv
from PyPDF2 import PdfReader
import pandas as pd

In [2]:
# Load environment variables
load_dotenv()

# Initialize Google Gemini API
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))

In [3]:
def get_pdf_text(pdf_paths):
    text = ""
    for pdf_path in pdf_paths:
        pdf_reader = PdfReader(pdf_path)
        for page in pdf_reader.pages:
            text += page.extract_text()
    return text

def get_txt_text(txt_paths):
    text = ""
    for txt_path in txt_paths:
        with open(txt_path, "r", encoding='utf-8') as file:
            text += file.read()
    return text

def get_csv_text(csv_paths):
    text = ""
    for csv_path in csv_paths:
        df = pd.read_csv(csv_path)
        text += df.to_string(index=False)
    return text

def get_combined_text(uploaded_files):
    combined_text = ""
    for file_path in uploaded_files:
        if file_path.endswith(".pdf"):
            combined_text += get_pdf_text([file_path])
        elif file_path.endswith(".txt"):
            combined_text += get_txt_text([file_path])
        elif file_path.endswith(".csv"):
            combined_text += get_csv_text([file_path])
    return combined_text

In [4]:
def get_gemini_response(query, context=None):
    try:
        model = genai.GenerativeModel('gemini-pro')
        if context:
            prompt=f"Answer the question based on this context: {context}. Question: {query}"
            response = model.generate_content([prompt, query])
        else:
            prompt=f"Answer the question using information from the internet. Question: {query}"
            response = model.generate_content([prompt, query])
        return response.text
    except Exception as e:
        print(f"Error generating response: {e}")
        return None

In [5]:
import json

def history(file_path, question, answer):
    """
    Appends a question-answer pair to a JSON file containing a list of dictionaries.
    
    Parameters:
    - file_path: Path to the JSON file.
    - question: The question to be added.
    - answer: The answer to be added.
    """
    # Load existing data from the JSON file
    try:
        with open(file_path, 'r') as file:
            data = json.load(file)
            if not isinstance(data, list):
                raise ValueError("JSON file does not contain a list of dictionaries.")
    except FileNotFoundError:
        # If the file does not exist, initialize an empty list
        data = []
    except json.JSONDecodeError:
        # Handle the case where the JSON file is empty or corrupted
        data = []
    except ValueError as e:
        print(e)
        return

    # Append the new question-answer pair to the list
    data.append({'question': question, 'answer': answer})

    # Write the updated list back to the JSON file
    try:
        with open(file_path, 'w') as file:
            json.dump(data, file, indent=4)
    except Exception as e:
        print(e)

In [6]:
def main():
    print("Welcome to AI Chatbot CLI")
    
    # Get file paths from user
    uploaded_files = input("Enter the paths of your files (PDF, TXT, CSV) separated by commas: ").split(',')
    uploaded_files = [file_path.strip() for file_path in uploaded_files if os.path.exists(file_path)]
    
    # Get user question
    user_question = input("Ask a question: ")
    
    # Generate response
    if user_question:
        print(f"\nYou: {user_question}")
        if uploaded_files:
            context_text = get_combined_text(uploaded_files)
            response = get_gemini_response(user_question, context=context_text)
        else:
            response = get_gemini_response(user_question)
        
        history("history.json", user_question, response)
        print(f"Chat Bot: {response}")

In [7]:
if __name__ == '__main__':
    main()

Welcome to AI Chatbot CLI

You: write a java program to filp all bits of an integer
Chat Bot: ```java
public class FlipBits {

    public static void main(String[] args) {
        int num = 12;
        System.out.println("Original number: " + num);

        // Flip all bits of the number using XOR with -1
        int flippedNum = num ^ -1;

        System.out.println("Flipped number: " + flippedNum);
    }
}
```
